#### Рекомендательные системы



Еще одна распространенная задача, связанная с аналитической обработкой данных, заключается в генерировании разного рода рекомендаций. Компания Netflix рекомендует фильмы, которые вы, возможно, захотите посмотреть, Amazon — товары, которые вы можете захотеть приобрести, Twitter — пользователей, за которыми вы могли бы последовать. В этой главе мы обратимся к нескольким способам использования данных для предоставления рекомендаций.
В частности, обратимся к набору данных users interests о сферах интересов пользователей, который использовался ранее:

In [1]:
import math, random
import numpy as np
from collections import defaultdict, Counter
from numpy import dot

In [2]:
# темы, интересующие пользователей 

In [2]:
users_interests = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

При этом подумаем над вопросом генерирования для пользователя предложений относительно новых тем на основе тем, интересующих его в данное время.

#### Рекомендации популярных тем

До появления Интернета за советом по поводу книг обычно ходили в библиотеку. Там библиотекарь был готов предложить книги, которые имеют отношение к сфере интересов читателя, либо аналогичные книги по желанию.
Учитывая ограниченное число пользователей DataSciencester и интересующих тем, нетрудно провести полдня, вручную раздавая рекомендации каждому пользователю по поводу новых тем. Но этот метод не очень хорошо поддается масштабированию, и ограничен личными познаниями и воображением, (Здесь нет ни капли намека на ограниченность чьих-то личных познаний и воображения.) Посмотрим, что можно сделать при помощи данных.

Простейший подход заключается в рекомендации на основе популярности тем:

In [3]:
popular_interests = Counter(interest
                            for user_interests in users_interests
                            for interest in user_interests).most_common()

которые выглядят так:

In [4]:
popular_interests

[('Python', 4),
 ('R', 4),
 ('Big Data', 3),
 ('HBase', 3),
 ('Java', 3),
 ('statistics', 3),
 ('regression', 3),
 ('probability', 3),
 ('Hadoop', 2),
 ('Cassandra', 2),
 ('MongoDB', 2),
 ('Postgres', 2),
 ('scikit-learn', 2),
 ('statsmodels', 2),
 ('pandas', 2),
 ('machine learning', 2),
 ('libsvm', 2),
 ('C++', 2),
 ('neural networks', 2),
 ('deep learning', 2),
 ('artificial intelligence', 2),
 ('Spark', 1),
 ('Storm', 1),
 ('NoSQL', 1),
 ('scipy', 1),
 ('numpy', 1),
 ('decision trees', 1),
 ('Haskell', 1),
 ('programming languages', 1),
 ('mathematics', 1),
 ('theory', 1),
 ('Mahout', 1),
 ('MapReduce', 1),
 ('databases', 1),
 ('MySQL', 1),
 ('support vector machines', 1)]

После их вычисления может оказаться, что предложенные пользователю наиболее популярные темы его уже не интересуют:

In [5]:
# наиболее популярные новые темы

In [7]:
def most_popular_new_interests(user_interests, max_results=5):
    suggestions = [(interest, frequency)
                   for interest, frequency in popular_interests
                   if interest not in user_interests]
    return suggestions[:max_results]

Так, для пользователя 1, чья сфера интересов лежит в:

In [5]:
users_interests[1]

['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres']

рекомендация пяти новых тем будет выглядеть следующим образом:

In [8]:
most_popular_new_interests(users_interests[1], max_results=5)

[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]

Для пользователя 3, который уже интересуется многими из ниже следующих тем, будет получено:

In [9]:
users_interests[3]

['R', 'Python', 'statistics', 'regression', 'probability']

In [10]:
most_popular_new_interests(users_interests[3], max_results=5)

[('Big Data', 3), ('HBase', 3), ('Java', 3), ('Hadoop', 2), ('Cassandra', 2)]

Естественно, маркетинговый ход в стиле "многие интересуются языком Python, так, может, и вам тоже стоит" не самый убедительный. Если на сайт зашел новичок, о котором ничего неизвестно, то, возможно, это было бы лучшее, что можно сде
лать. Посмотрим, каким образом можно усовершенствовать рекомендации каждому пользователю, если основываться на интересующих его темах.

#### Коллаборативная фильтрация на основе пользователя

Один из способов учесть сферу интересов пользователя заключается в том, чтобы отыскать пользователей, которые на него как-то похожи, и затем предложить темы, в которых эти пользователи заинтересованы.
Для этого потребуется метод измерения степени сходства двух пользователей. Здесь будет использована метрика, называемая косинусным коэффициентом подобия (cosine similarity). Пусть даны два вектора, v и w. Тогда косинусный коэффициент определяется следующим образом:

In [11]:
#
# косинусный коэффициент подобия 
#

def cosine_similarity(v, w):
    return dot(v, w) / math.sqrt(dot(v, v) * dot(w, w))

In [12]:
[1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[1, 0, 0 ,0 ,0, 0 ,0, 0, 1, 0]

[1, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [17]:
cosine_similarity([1, 0, 0, 0, 0, 0, 1, 0, 1, 1],[1, 0, 0 ,0 ,0, 0 ,0, 0, 1, 1])

0.8660254037844387

Он измеряет "угол" между векторами v и w. Если векторы v и w — равнонаправлены, то числитель и знаменатель равны, и их косинусный коэффициент равен 1. Если они противоположно направленные, то их косинусный коэффициент равен -1. И если вектор v нулевой, а вектор w — нет (и наоборот), то результат dot (v, w) равен О, и соответственно их косинусный коэффициент равен 0.
Применим его к векторам, состоящим из 0 и 1, где каждый вектор v обозначает темы, интересующие одного пользователя, т. е. его сферу интересов. Элемент v[i] равен 1, если пользователь указал i-ю тему, и 0 — в противном случае. Соответственно, под "сходством пользователей" будут пониматься "пользователи, чьи векторы интересующих их тем наиболее близко указывают в том же направлении". У людей с одинаковыми интересами коэффициент подобия будет равен 1, при отсутствии одинаковых интересов коэффициент подобия будет равен 0. В противном случае коэффициент подобия попадет в интервал между этими числами, где число, близкое к 1, будет указывать на большое "сходство", а число, близкое к 0, — на небольшое.
Неплохо начать с создания списка известных тем и (неявного) присваивания им индексов. Это можно сделать, воспользовавшись генератором последовательности в виде множества, которое позволяет получить перечень тем без повторов, затем преобразовав полученный результат в список и упорядочив его. Первая тема в полученном списке будет с индексом 0 и т. д.:

In [22]:
# список интересующих тем без повторов 

In [18]:
unique_interests = sorted(list({ interest
                                 for user_interests in users_interests
                                 for interest in user_interests }))

В итоге получится список, который начинается со следующих записей:

In [19]:
unique_interests

['Big Data',
 'C++',
 'Cassandra',
 'HBase',
 'Hadoop',
 'Haskell',
 'Java',
 'Mahout',
 'MapReduce',
 'MongoDB',
 'MySQL',
 'NoSQL',
 'Postgres',
 'Python',
 'R',
 'Spark',
 'Storm',
 'artificial intelligence',
 'databases',
 'decision trees',
 'deep learning',
 'libsvm',
 'machine learning',
 'mathematics',
 'neural networks',
 'numpy',
 'pandas',
 'probability',
 'programming languages',
 'regression',
 'scikit-learn',
 'scipy',
 'statistics',
 'statsmodels',
 'support vector machines',
 'theory']

Далее для каждого пользователя следует создать вектор интересующих его тем из О и 1. Для этого надо лишь просмотреть список уникальных тем unique interests, назначая 1, если пользователь заинтересован в теме, и 0 — если нет:

In [7]:
# создать вектор интересующих пользователя 

In [20]:
# создаем список интересующих пользователя тем
def make_user_interest_vector(user_interests):
    """при заданном списке интересующих пользователя тем создать вектор,
    чей i-й элемент равен 1, если unique_interests[i] есть в списке,
    и 0 в противном случае"""
    return [1 if interest in user_interests else 0
            for interest in unique_interests]

Затем можно создать матрицу сфер интересов пользователей, отобразив эту функцию на каждый элемент списка списков интересующих тем (т. е. применив ее к каждому элементу списка при помощи функции map:

In [8]:
#	матрица сфер интересов в формате (пользователь, интересующие темы),
#	где список тем для каждого пользователя преобразован в 0 и 1 

In [21]:
user_interest_matrix = list(map(make_user_interest_vector, users_interests))

In [27]:
users_interests

[['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra'],
 ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres'],
 ['Python', 'scikit-learn', 'scipy', 'numpy', 'statsmodels', 'pandas'],
 ['R', 'Python', 'statistics', 'regression', 'probability'],
 ['machine learning', 'regression', 'decision trees', 'libsvm'],
 ['Python', 'R', 'Java', 'C++', 'Haskell', 'programming languages'],
 ['statistics', 'probability', 'mathematics', 'theory'],
 ['machine learning', 'scikit-learn', 'Mahout', 'neural networks'],
 ['neural networks', 'deep learning', 'Big Data', 'artificial intelligence'],
 ['Hadoop', 'Java', 'MapReduce', 'Big Data'],
 ['statistics', 'R', 'statsmodels'],
 ['C++', 'deep learning', 'artificial intelligence', 'probability'],
 ['pandas', 'R', 'Python'],
 ['databases', 'HBase', 'Postgres', 'MySQL', 'MongoDB'],
 ['libsvm', 'regression', 'support vector machines']]

In [43]:
np.array(user_interest_matrix)

array([[1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],


In [45]:
np.array(user_interest_matrix).shape

(15, 36)

Теперь элемент матрицы user interest matrixt[i][j] равен 1, если пользователь i указал заинтересованность в теме j, и 0 — в противном случае.
Поскольку набор данных небольшой, вычислить попарные коэффициенты подобия между всеми пользователями не составляет труда:

In [9]:
#	матрица сходств между пользователями

In [25]:
user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j)
                      for interest_vector_j in user_interest_matrix]
                     for interest_vector_i in user_interest_matrix]

In [29]:
users_interests[0]

['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra']

In [31]:
users_interests[9]

['Hadoop', 'Java', 'MapReduce', 'Big Data']

In [40]:
user_similarities[9]

[0.5669467095138409,
 0.0,
 0.0,
 0.0,
 0.0,
 0.20412414523193154,
 0.0,
 0.0,
 0.25,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [42]:
np.array(user_similarities).shape

(15, 15)

In [34]:
users_interests[0]

['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra']

In [ ]:
enumerate(user_similarities[user_id])

In [35]:
users_interests[9]

['Hadoop', 'Java', 'MapReduce', 'Big Data']

Теперь элемент матрицы user similarities[i][j] будет показывать коэффициент подобия между пользователями i и j.
Например, элемент матрицы user similarities [0] [9] равен 0.57, поскольку оба пользователя разделяют заинтересованность в Hadoop, Java и Big Data. С другой стороны, элемент матрицы user similarities [0] [8] равен всего 0.19, поскольку пользователи 0 и 8 имеют всего одну общую заинтересованность, а именно в Big Data.
В данном случае user_simiiarities[i] — это вектор коэффициентов подобия пользователя i с остальными пользователями. Этим вектором можно воспользоваться для написания функции, которая для заданного пользователя находит наиболее похожих пользователей. При этом, разумеется, ни сам пользователь, ни пользователи с нулевым коэффициентом не рассматриваются, а результаты сортируются по убыванию от наиболее до наименее похожих:

In [11]:
# пользователи наиболее похожие на пользователя user__id 

In [32]:
def most_similar_users_to(user_id):
    pairs = [(other_user_id, similarity)                      # find other
             for other_user_id, similarity in                 # users with
                enumerate(user_similarities[user_id])         # nonzero
             if user_id != other_user_id and similarity > 0]  # similarity

    return sorted(pairs,                                      # sort them
                  key=lambda pair: pair[1],                   # most similar
                  reverse=True)                               # first

К примеру, если вызовем most_similar_users_to(0), то получим:

In [33]:
most_similar_users_to(0)

[(9, 0.5669467095138409),
 (1, 0.3380617018914066),
 (8, 0.1889822365046136),
 (13, 0.1690308509457033),
 (5, 0.1543033499620919)]

Каким образом следует использовать этот результат для рекомендации пользователю новых тем? По каждой теме можно попросту просуммировать коэффициенты подобия других пользователей, которые интересуются данной темой:

In [12]:
#	рекомендации на основе пользователя

In [34]:
def user_based_suggestions(user_id, include_current_interests=False):
    # sum up the similarities
    suggestions = defaultdict(float)
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity

    # convert them to a sorted list
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)

    # and (maybe) exclude already-interests
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]

In [35]:
suggestions = defaultdict(float)
for other_user_id, similarity in most_similar_users_to(0):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity

In [36]:
suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)


In [37]:
suggestions

[('Big Data', 0.7559289460184544),
 ('Java', 0.7212500594759328),
 ('Hadoop', 0.5669467095138409),
 ('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('HBase', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('Cassandra', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

In [53]:
[(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[0]]

[('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

In [ ]:

0  1                [1,0,1,1,0,1]
1    0.58           [1,0,1,1,1,1]         
2         0.2       [1,1,0,0,0,1]
3               0.1 [0,1,0,1,0,0]         




In [39]:
user_based_suggestions(0)

[('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

Если вызвать user based suggestions (0), то несколько первых рекомендуемых тем из всего списка будут следующими:

In [48]:
user_based_suggestions(0)

[('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

Выглядит довольно приличной рекомендацией для того, чьи интересы лежат в области Big Data и связаны с базами данных. (Веса не имеют какого-то особого смысла; они используются всего лишь для упорядочивания.)
Этот подход перестает справляться со своей задачей, когда число элементов становится очень большим. Напомним, что в связи с проблемой проклятия размерности, в многомерных векторных пространствах большинство векторов расположены друг от друга на большом расстоянии (и поэтому указывают в совершенно разных направлениях). Другими словами, при наличии большого количества интересующих тем "наиболее похожие пользователи" могут оказаться совсем непохожими на конкретного пользователя.
Возьмем сайт наподобие Amazon.com. Можно попытаться установить похожих на меня пользователей на основе присущей мне модели покупательского поведения, но, скорее всего, во всем мире нет никого, чья покупательская история выглядит даже отдаленно похожей на мою. Кто бы ни был этот "наиболее похожий" на меня покупатель, он, вероятно, совсем не похож на меня, и, как результат, его покупки почти наверняка будут содействовать отвратительным рекомендациям.

#### Коллаборативная фильтрация по схожести предметов

Альтернативный подход заключается в вычислении сходства непосредственно между интересующими темами. После этого можно сгенерировать рекомендации по каждому пользователю путем агрегирования тем, похожих на те, которые интересуют его в данное время.
Для начала следует транспонировать матрицу сфер интересов пользователей, благодаря чему строки соответствуют темам, а столбцы — пользователям:

In [46]:
#
# Item-Based Collaborative Filtering
#

interest_user_matrix = [[user_interest_vector[j]
                         for user_interest_vector in user_interest_matrix]
                        for j, _ in enumerate(unique_interests)]

In [48]:
np.array(interest_user_matrix).shape

(36, 15)

Как выглядит матрица групп пользователей по интересам interest user matrix? Строка j матрицы— это столбец j матрицы сфер интересов пользователей user interest matrix. Иными словами, она содержит 1 для пользователя, у которого есть интерес к данной теме, и 0 для пользователя, у которого его нет.

Например, unique_interests[0] равен Big Data, значит, interest_user_matrix[0] равен: 

In [63]:
unique_interests[0]

'Big Data'

In [64]:
interest_user_matrix[0]

[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]

In [65]:
unique_interests

['Big Data',
 'C++',
 'Cassandra',
 'HBase',
 'Hadoop',
 'Haskell',
 'Java',
 'Mahout',
 'MapReduce',
 'MongoDB',
 'MySQL',
 'NoSQL',
 'Postgres',
 'Python',
 'R',
 'Spark',
 'Storm',
 'artificial intelligence',
 'databases',
 'decision trees',
 'deep learning',
 'libsvm',
 'machine learning',
 'mathematics',
 'neural networks',
 'numpy',
 'pandas',
 'probability',
 'programming languages',
 'regression',
 'scikit-learn',
 'scipy',
 'statistics',
 'statsmodels',
 'support vector machines',
 'theory']

потому что пользователи 0, 8 и 9 указали свою заинтересованность в Big Data.
Теперь можно снова воспользоваться косинусным коэффициентом подобия. Если ровно те же самые пользователи заинтересованы в двух темах, то их сходство будет равно 1. Если нет двух пользователей, которых интересуют обе темы, то их коэффициентподобия будет равен 0.

In [13]:
#	сходство интересов

In [49]:
interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j)
                          for user_vector_j in interest_user_matrix]
                         for user_vector_i in interest_user_matrix]

Например, при помощи следующей функции можно найти темы, максимально похожие на тему Big Data (тема 0):

In [14]:
#	темы, максимально похожие на тему interest_id 

In [50]:
def most_similar_interests_to(interest_id):
    similarities = interest_similarities[interest_id]
    pairs = [(unique_interests[other_interest_id], similarity)
             for other_interest_id, similarity in enumerate(similarities)
             if interest_id != other_interest_id and similarity > 0]
    return sorted(pairs,
                  key=lambda pair: pair[1],
                  reverse=True)

Функция сгенерирует следующие похожие темы:

In [51]:
most_similar_interests_to(0)

[('Hadoop', 0.8164965809277261),
 ('Java', 0.6666666666666666),
 ('MapReduce', 0.5773502691896258),
 ('Spark', 0.5773502691896258),
 ('Storm', 0.5773502691896258),
 ('Cassandra', 0.4082482904638631),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('HBase', 0.3333333333333333)]

Теперь можно сгенерировать для пользователя рекомендации, просуммировав коэффициенты подобия тем, которые похожи на интересующие его темы:

In [ ]:
# рекомендации на основе предметов

In [52]:
def item_based_suggestions(user_id, include_current_interests=False):
    #	список рекомендаций, где суммируются похожие темы 
    suggestions = defaultdict(float)
    #	вектор тем, интересующих пользователя user_id 
    user_interest_vector = user_interest_matrix[user_id]
    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            # темы, похожие на тему interest_id
            similar_interests = most_similar_interests_to(interest_id)
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity
#	упорядочить по весу
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)

    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]

Для пользователя 0 эта функция сгенерирует следующие (на вид разумные) рекомендации:

In [70]:
if __name__ == "__main__":

    print("Popular Interests")
    print(popular_interests)
    print()

    print("Most Popular New Interests")
    print("already like:", ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"])
    print(most_popular_new_interests(["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"]))
    print()
    print("already like:", ["R", "Python", "statistics", "regression", "probability"])
    print(most_popular_new_interests(["R", "Python", "statistics", "regression", "probability"]))
    print()

    print("User based similarity")
    print("most similar to 0")
    print(most_similar_users_to(0))

    print("Suggestions for 0")
    print(user_based_suggestions(0))
    print()

    print("Item based similarity")
    print("most similar to 'Big Data'")
    print(most_similar_interests_to(0))
    print()

    print("suggestions for user 0")
    print(item_based_suggestions(0))

Popular Interests
[('Python', 4), ('R', 4), ('Big Data', 3), ('HBase', 3), ('Java', 3), ('statistics', 3), ('regression', 3), ('probability', 3), ('Hadoop', 2), ('Cassandra', 2), ('MongoDB', 2), ('Postgres', 2), ('scikit-learn', 2), ('statsmodels', 2), ('pandas', 2), ('machine learning', 2), ('libsvm', 2), ('C++', 2), ('neural networks', 2), ('deep learning', 2), ('artificial intelligence', 2), ('Spark', 1), ('Storm', 1), ('NoSQL', 1), ('scipy', 1), ('numpy', 1), ('decision trees', 1), ('Haskell', 1), ('programming languages', 1), ('mathematics', 1), ('theory', 1), ('Mahout', 1), ('MapReduce', 1), ('databases', 1), ('MySQL', 1), ('support vector machines', 1)]

Most Popular New Interests
already like: ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres']
[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]

already like: ['R', 'Python', 'statistics', 'regression', 'probability']
[('Big Data', 3), ('HBase', 3), ('Java', 3), ('Hadoop', 2), ('Cassandra', 2)]

User b

### Для дальнейшего изучения

♦	Crab (http://muricoca.github.io/crab)— это прикладная среда для создания рекомендательных систем (recommender systems) на языке Python.

♦	Библиотека Graphlab также располагает инструментарием для создания реко-мендательных систем (https://turi.com/products/create/docs/graphlab.toolkits. recommender.html).

♦	Премия Netflix (http://www.netflixprize.com) — это достаточно известный конкурс по созданию усовершенствованной системы предложения рекомендаций о фильмах пользователям сети потокового мультимедиа Netflix.

In [ ]:
#https://realpython.com/build-recommendation-engine-collaborative-filtering/

In [ ]:
#https://github.com/grahamjenson/list_of_recommender_systems

In [ ]:
#https://arxiv.org/pdf/2212.04782.pdf

In [ ]:
#https://arxiv.org/pdf/2212.04481.pdf

In [ ]:
#https://github.com/lyst/lightfm

---